
Facial Recognition using PyTorch and OpenCV

https://ritik12.medium.com/facial-recognition-using-pytorch-and-opencv-467c4e41d1f


Machine Learning - Face Recognition CNN Pytorch.ipynb
https://github.com/rubencg195/Pytorch-Tutorials/blob/master/Machine%20Learning%20-%20Face%20Recognition%20CNN%20Pytorch.ipynb



Face Recognition Using Pytorch
https://github.com/timesler/facenet-pytorch

Face Landmarks Detection With PyTorch

https://towardsdatascience.com/face-landmarks-detection-with-pytorch-4b4852f5e9c4



다중입력 deep neural network
https://rosenfelder.ai/multi-input-neural-network-pytorch/



Understanding dimensions in PyTorch
https://towardsdatascience.com/understanding-dimensions-in-pytorch-6edf9972d3be

학습하기 
https://github.com/deeplearningzerotoall/PyTorch

In [1]:
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor
from FaceFeatureDataset import FaceFeatureDataset
import dlib_index

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [2]:
batch_size = 30
epochs = 5

In [3]:
# Create DataLoader 
training_data = FaceFeatureDataset(feature_file="./outimg/Train/facefeature.csv", label_file="./Dataset/Train/csv/train.csv")
test_data = FaceFeatureDataset(feature_file="./outimg/Test/facefeature.csv", label_file="./Dataset/Test/csv/test.csv")

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# 데이터 로드 확인 
for X, y in train_loader:
    print(f"Shape of X [N, F, C]: {X.shape}") # N , Channel, H= width W = height
    print(f"Shape of Tensor y: {y.shape} {y.dtype}")       
    break

n_total_steps = len(train_loader)
# print(f'Traing dat length {n_total_steps}')
# print(y)
print('X type', X.dtype)
print('y type', y.dtype)


[[-0.35331735 -0.23438239]
 [-0.3455863  -0.14884205]
 [-0.32834524 -0.06336822]
 [-0.30871004  0.0244665 ]
 [-0.28431982  0.11226797]
 [-0.23856528  0.19516478]
 [-0.17390709  0.26128605]
 [-0.0974445   0.31543666]
 [-0.00936039  0.33146405]
 [ 0.07854083  0.32133886]
 [ 0.16381516  0.27556774]
 [ 0.23469144  0.21087629]
 [ 0.28408703  0.13444695]
 [ 0.3167736   0.04862396]
 [ 0.342311   -0.03952665]
 [ 0.3630934  -0.127644  ]
 [ 0.37434915 -0.21807237]
 [-0.2918846  -0.28949732]
 [-0.24689487 -0.31596577]
 [-0.18986797 -0.32111982]
 [-0.13508557 -0.30723718]
 [-0.08259755 -0.28145036]
 [ 0.07669535 -0.2825643 ]
 [ 0.13595012 -0.30913252]
 [ 0.20005965 -0.32146895]
 [ 0.26189145 -0.31952372]
 [ 0.31205183 -0.28658777]
 [ 0.00113056 -0.20832959]
 [ 0.00156284 -0.14651443]
 [-0.00039902 -0.08706015]
 [-0.00234425 -0.02522836]
 [-0.05915501  0.01083325]
 [-0.03055844  0.02014377]
 [ 0.00041565  0.02943765]
 [ 0.03125672  0.01971149]
 [ 0.05494865  0.0076577 ]
 [-0.22707677 -0.20197849]
 

In [4]:

num_classes = 33

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(68 * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
            #nn.ReLU(),
        )
    

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=136, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=33, bias=True)
  )
)


In [5]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)        
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print('batch',  batch)
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
# Training
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)    
print("Done!")

Epoch 1
-------------------------------
[[-3.56650144e-01 -2.15680629e-01]
 [-3.47696990e-01 -1.27026036e-01]
 [-3.31577986e-01 -3.59266065e-02]
 [-3.13047856e-01  5.27954362e-02]
 [-2.89746106e-01  1.43945456e-01]
 [-2.44828656e-01  2.25670218e-01]
 [-1.80672944e-01  2.95558631e-01]
 [-1.04444832e-01  3.51165891e-01]
 [-1.84036698e-02  3.73321086e-01]
 [ 7.02677891e-02  3.61973703e-01]
 [ 1.52026281e-01  3.12267780e-01]
 [ 2.24308953e-01  2.48128936e-01]
 [ 2.77521938e-01  1.71883956e-01]
 [ 3.11665267e-01  8.35328549e-02]
 [ 3.36265326e-01 -9.67418775e-03]
 [ 3.58471125e-01 -1.02898091e-01]
 [ 3.71099919e-01 -1.96189433e-01]
 [-3.01127195e-01 -2.79937506e-01]
 [-2.55383581e-01 -3.15530866e-01]
 [-1.97854206e-01 -3.24703187e-01]
 [-1.40459701e-01 -3.14721555e-01]
 [-8.55606273e-02 -2.90391266e-01]
 [ 7.72650614e-02 -2.91638970e-01]
 [ 1.37272999e-01 -3.12765688e-01]
 [ 2.01985121e-01 -3.21887434e-01]
 [ 2.64201850e-01 -3.16660553e-01]
 [ 3.14211249e-01 -2.77998507e-01]
 [ 1.43317637e-

In [8]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    print('test size', size )
    # num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            print("X Shape", X.shape)
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)            
            print('pred =', pred)
            #print('loss', loss)
            #print('real', y)



In [10]:

test(test_loader, model, criterion)

print("Done !")

test size 100
[[-3.84834349e-01 -2.03853011e-01]
 [-3.73717993e-01 -1.04609154e-01]
 [-3.55502248e-01 -7.67709874e-03]
 [-3.34909111e-01  8.69103521e-02]
 [-3.04871976e-01  1.81563392e-01]
 [-2.48781890e-01  2.64591783e-01]
 [-1.73672661e-01  3.28863382e-01]
 [-9.14148614e-02  3.83740187e-01]
 [-6.55012764e-03  4.03218389e-01]
 [ 7.61831477e-02  3.89626265e-01]
 [ 1.63884357e-01  3.40651989e-01]
 [ 2.42223561e-01  2.79807121e-01]
 [ 3.01773161e-01  2.04665095e-01]
 [ 3.37827563e-01  1.12832136e-01]
 [ 3.62076968e-01  2.09172014e-02]
 [ 3.84030968e-01 -8.04581195e-02]
 [ 3.94179970e-01 -1.81915417e-01]
 [-3.11118782e-01 -2.78896660e-01]
 [-2.61259109e-01 -3.18689287e-01]
 [-1.95085600e-01 -3.27674210e-01]
 [-1.33781642e-01 -3.15442920e-01]
 [-7.73308575e-02 -2.84356475e-01]
 [ 7.61667565e-02 -2.88012743e-01]
 [ 1.42471433e-01 -3.15885603e-01]
 [ 2.08661333e-01 -3.27231526e-01]
 [ 2.77048260e-01 -3.14951062e-01]
 [ 3.23973089e-01 -2.72125185e-01]
 [ 4.77937981e-03 -2.08230704e-01]
 [ 1.9

In [11]:


x_ = np.array([[
    [-0.36369685665697504, -0.21546217565417758], [-0.3574112677601623, -
                                                   0.12973942618903955], [-0.34360908442591476, -0.041386801250754224],
    [-0.32728893521312186, 0.047003126885731494], [-0.3008596192879478,
                                                   0.13302430193647485], [-0.24910143178451916, 0.20934664545504422],
    [-0.1820489330188162, 0.27330297877009146], [-0.1047753579461298,
                                                 0.32733666136376105], [-0.016907674584453414, 0.3462680344506007],
    [0.0714076471556312, 0.3349838169948983], [0.15768994459377905,
                                               0.290928739919908], [0.23164353502660584, 0.22906138570409784],
    [0.28812057710241923, 0.15686104558670244], [0.32215974546052994,
                                                 0.06921718141423006], [0.3461270503044599, -0.018575895551044996],
    [0.3676136924680454, -0.1089242415930659], [0.3790284711174502, -
                                                0.19942180042788948], [-0.29962861374732563, -0.29006857205551595],
    [-0.25378298315870307, -0.3246486367875362], [-0.19323989247901685, -
                                                  0.3313072576663558], [-0.13799385594382574, -0.3204147237917603],
    [-0.08297163859783863, -0.2944143946458938], [0.0857693784628889, -
                                                  0.2944330462449941], [0.146536288331779, -0.3161994623950847],
    [0.20970925448461253, -0.330374677711339], [0.27513906412858635, -
                                                0.3268868286795764], [0.32741949640682433, -0.2858160074606396],
    [0.00400076800702176, -0.21505184047396997], [0.0031054912502057608, -
                                                  0.1546206593888858], [-0.0002704481869548747, -0.09674474738054728],
    [-0.0037209940205167324, -0.033832903615118703], [-0.07230292391244719,
                                                      0.005447364090185869], [-0.03971858028416264, 0.01600416918097547],
    [-0.004616270777332843, 0.02659827746996546], [0.0307844643151024,
                                                   0.017048658730594135], [0.06366723352899217, 0.007461736793021978],
    [-0.2304684842832848, -0.19837731087827104], [-0.19228866092489982, -
                                                  0.22551538756925782], [-0.14192934335399632, -0.22476932360524426],
    [-0.10212683087388219, -0.19143891601294627], [-0.14767403587689942, -
                                                   0.17700257830928734], [-0.19803335344780293, -0.17774864227330067],
    [0.10182840528827686, -0.1884173569586921], [0.14263810411980915, -
                                                 0.22307202808711402], [0.19558999396565901, -0.22732459268199012],
    [0.23802238191892033, -0.20151077952712704], [0.1999171649569368, -
                                                  0.17940863459323053], [0.1470025783092873, -0.17767403587689945],
    [-0.137005321191508, 0.12285918042679256], [-0.08882824071534368,
                                                0.10090624828569872], [-0.04324373251412583, 0.08395194470349432],
    [-0.008104119809095423, 0.09202808711393928], [0.027259312085139098,
                                                   0.0849964342531132], [0.07483954139009275, 0.10333095616874222],
    [0.1299363651324813, 0.12429535355751808], [0.07649953371002238,
                                                0.16128147457348174], [0.030989631905205983, 0.17319984639859565],
    [-0.006854462669372952, 0.17767623018267586], [-0.04706731032969447,
                                                   0.17204344725437493], [-0.08961160787755773, 0.15378353173514736],
    [-0.11182566240605629, 0.12323221240879911], [-0.04624663996927969,
                                                  0.11664819792638104], [-0.008514454989302811, 0.11972571177793634],
    [0.026774370508530265, 0.11772999067420054], [0.10723736902737391,
                                                  0.1264775906522574], [0.0292550331888749, 0.1202852597509464],
    [-0.006071095507158897, 0.12479894673322722], [-0.043765977288935165, 0.1192034670031269]]], dtype=np.float32)

print(x_.shape, x_.dtype)
model.eval()
x = torch.Tensor(x_).to(device)
with torch.no_grad():
    pred = model(x)
    print(f'{pred} {pred.shape}')


(1, 68, 2) float32
tensor([[-0.0996,  0.0320,  0.0752, -0.0613, -0.0231,  0.0487, -0.1100,  0.0559,
          0.1010,  0.0740, -0.0202, -0.0029, -0.0938, -0.0140, -0.0360, -0.0505,
         -0.0338, -0.0180, -0.0573,  0.0878, -0.0500, -0.0203, -0.0664,  0.0884,
          0.0692, -0.0079,  0.0629,  0.0778,  0.0804, -0.0583, -0.0260,  0.1179,
          0.0240]], device='cuda:0') torch.Size([1, 33])
